In [1]:
import os
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from scripts.functions import Tables, run_sql, revise_table, add_tables_safely
import scripts.config as cfg

# Create SQL Tables

In [2]:
sql_drop_table = '''
    DROP TABLE IF EXISTS demographic, debt, payment;
    '''
run_sql(sql_drop_table)

In [3]:
sql_demo = '''
      CREATE TABLE IF NOT EXISTS demographic (
        customer_id INTEGER Not Null PRIMARY KEY,
        age INTEGER,
        monthlyincome INTEGER,
        numberofdependents INTEGER
      );
      '''

sql_debt = '''
      CREATE TABLE IF NOT EXISTS debt (
        customer_id INTEGER Not Null PRIMARY KEY,
        seriousdlqin2yrs BOOLEAN,
        numberoftime30_59dayspastduenotworse INTEGER,
        numberoftime60_89dayspastduenotworse INTEGER,
        numberoftimes90dayslate INTEGER
      );
      '''

sql_payment = '''
      CREATE TABLE IF NOT EXISTS payment (
        customer_id INTEGER Not Null PRIMARY KEY,
        debtratio DOUBLE PRECISION,
        revolvingutilizationofunsecuredlines DOUBLE PRECISION,
        numberofopencreditlinesandloans INTEGER,
        numberrealestateloansorlines INTEGER
      );
      '''

for sql in [sql_demo, sql_debt, sql_payment]:
    run_sql(sql)

# Import Table and Split

In [5]:
table1 = Tables('../sample_data.csv')
table1.split_tables()

In [6]:
table1.demographic_table.head()

,customer_id,age,monthlyincome,numberofdependents
0,1,45,9120.0,2
1,2,40,2600.0,1
2,3,38,3042.0,0
3,4,30,3300.0,0
4,5,49,63588.0,0


In [7]:
table1.debt_table.head()

,customer_id,seriousdlqin2yrs,numberoftime30_59dayspastduenotworse,numberoftime60_89dayspastduenotworse,numberoftimes90dayslate
0,1,True,2,0,0
1,2,False,0,0,0
2,3,False,1,0,1
3,4,False,0,0,0
4,5,False,1,0,0


In [8]:
table1.payment_table.head()

,customer_id,debtratio,revolvingutilizationofunsecuredlines,numberofopencreditlinesandloans,numberrealestateloansorlines
0,1,0.802982,0.766127,13,6
1,2,0.121876,0.957151,4,0
2,3,0.085113,0.658180,2,0
3,4,0.036050,0.233810,5,0
4,5,0.024926,0.907239,7,1


# Append Tables to SQL

In [9]:
engine = create_engine(
    f'postgresql://{cfg.postgres["user"]}:'\
    f'{cfg.postgres["password"]}'\
    f'@{cfg.postgres["host"]}:'\
    f'{cfg.postgres["port"]}/{cfg.postgres["dbname"]}'
)

In [10]:
add_tables_safely(table_name='demographic', engine=engine, table=table1)
add_tables_safely(table_name='debt', engine=engine, table=table1)
add_tables_safely(table_name='payment', engine=engine, table=table1)

# View Table from SQL

In [11]:
demo_df = pd.read_sql(sql='SELECT * FROM demographic;', con=engine)
debt_df = pd.read_sql(sql='SELECT * FROM debt;', con=engine)
paym_df = pd.read_sql(sql='SELECT * FROM payment;', con=engine)

In [12]:
demo_df.head()

,customer_id,age,monthlyincome,numberofdependents
0,1,45,9120.0,2.0
1,2,40,2600.0,1.0
2,3,38,3042.0,0.0
3,4,30,3300.0,0.0
4,5,49,63588.0,0.0


In [13]:
debt_df.head()

,customer_id,seriousdlqin2yrs,numberoftime30_59dayspastduenotworse,numberoftime60_89dayspastduenotworse,numberoftimes90dayslate
0,1,True,2,0,0
1,2,False,0,0,0
2,3,False,1,0,1
3,4,False,0,0,0
4,5,False,1,0,0


In [14]:
paym_df.head()

,customer_id,debtratio,revolvingutilizationofunsecuredlines,numberofopencreditlinesandloans,numberrealestateloansorlines
0,1,0.802982,0.766127,13,6
1,2,0.121876,0.957151,4,0
2,3,0.085113,0.658180,2,0
3,4,0.036050,0.233810,5,0
4,5,0.024926,0.907239,7,1


# Add same data format to SQL

In [15]:
table2 = Tables('../sample_data.csv')
table2.split_tables()

In [16]:
# Before
demo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   customer_id         150000 non-null  int64  
 1   age                 150000 non-null  int64  
 2   monthlyincome       120269 non-null  float64
 3   numberofdependents  146076 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 4.6 MB


In [17]:
add_tables_safely(table_name='demographic', engine=engine, table=table2)

In [19]:
new_demo_df = pd.read_sql(sql='SELECT * FROM demographic;', con=engine)

In [20]:
# After
new_demo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   customer_id         300000 non-null  int64  
 1   age                 300000 non-null  int64  
 2   monthlyincome       240538 non-null  float64
 3   numberofdependents  292152 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 9.2 MB
